In [1]:
import warnings
warnings.filterwarnings('ignore')

from tqdm import tqdm
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

# import tensorflowjs as tfjs

from python.database import Database

Using TensorFlow backend.


In [2]:
db = Database()
query = 'SELECT doodle_id, tune_id FROM doodles'
lookup = dict(db.query(query))

doodles = pd.read_csv('data/doodle_images.csv').set_index(['row', 'column'])
tunes = pd.read_csv('data/music_images.csv').set_index(['row', 'column'])

query = 'SELECT note_id, note_name, note_octave FROM notes'
notes_df = pd.DataFrame(data=db.query(query), columns=['note_id', 'note_name', 'note_octave'])
notes_df['note'] = notes_df.note_name + notes_df.note_octave.astype(str)

In [3]:
idx = [i for i in doodles.index]
columns = []
for r, c in idx:
    for p in ['r', 'g', 'b']:
        s = 'r%i_c%i_%s' % (r, c, p)
        columns.append(s)
        
data = []
for d in tqdm(doodles.T.index):
    to_append = [d]
    pixels = doodles.T.loc[d]
    for p in pixels.tolist():
        p = p[1:-1].split(' ')
        for i in range(3):
            try:
                to_append.append(int(p[i]))
            except:
                to_append.append(0)
    data.append(to_append)

doodle_frame = pd.DataFrame(columns=['doodle_id'] + columns, data=data)
doodle_frame.set_index('doodle_id', inplace=True)

100%|██████████| 111/111 [00:00<00:00, 181.52it/s]


In [4]:
idx = [i for i in tunes.index]
columns = []
for r, c in idx:
    s = 'r%i_c%i' % (r, c)
    columns.append(s)
        
data = []
for t in tqdm(tunes.T.index):
    to_append = [t]
    pixels = tunes.T.loc[t]
    for p in pixels.tolist():
        p = eval(p)
        if p == 0:
            to_append.append(0)
        else:
            to_append.append(1)
                
    data.append(to_append)
    
tune_frame = pd.DataFrame(columns=['tune_id'] + columns, data=data)
tune_frame.set_index('tune_id', inplace=True)

100%|██████████| 2001/2001 [00:24<00:00, 80.10it/s]


In [5]:
X = doodle_frame.values
y = []

l = set(lookup.keys())
t = set(doodles.T.index.tolist())

diff = l.difference(t)

for d, t in lookup.items():
    if d not in diff:
        y.append(tune_frame.iloc[t].tolist())

import numpy as np
y = np.asarray(y)

print('X shape:', X.shape)
print('y shape:', y.shape)

X shape: (111, 2352)
y shape: (111, 160)


In [35]:
X

array([[ 0.15932892,  0.15824994,  0.21718612, ...,  0.27386516,
         0.31465839,  0.31465839],
       [ 0.15932892,  0.15824994,  0.21718612, ...,  0.27386516,
         0.31465839,  0.31465839],
       [ 0.15932892,  0.15824994,  0.21718612, ...,  0.27386516,
         0.31465839,  0.31465839],
       ...,
       [ 0.15932892,  0.15824994,  0.21718612, ...,  0.27386516,
         0.31465839,  0.31465839],
       [-5.99215281, -5.98976014, -4.60434577, ...,  0.27386516,
         0.31465839,  0.31465839],
       [ 0.15932892,  0.15824994,  0.21718612, ...,  0.27386516,
         0.31465839,  0.31465839]])

In [6]:
from sklearn.preprocessing import StandardScaler

X = StandardScaler().fit_transform(X)

In [25]:
model = Sequential()
model.add(Dense(512, input_shape=(X.shape[1],), activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(optimizer=Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 512)               1204736   
_________________________________________________________________
dense_16 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_17 (Dense)             (None, 256)               131328    
_________________________________________________________________
dense_18 (Dense)             (None, 160)               41120     
Total params: 1,639,840
Trainable params: 1,639,840
Non-trainable params: 0
_________________________________________________________________


In [37]:
notes_df

,note_id,note_name,note_octave,note
0,1,Bb,3,Bb3
1,2,B,3,B3
2,3,Cb,4,Cb4
3,4,C,4,C4
4,5,C#,4,C#4
5,6,Db,4,Db4
6,7,D,4,D4
7,8,D#,4,D#4
8,9,Eb,4,Eb4
9,10,E,4,E4


In [26]:
Xm1 = np.concatenate((X[:-1], X[:-1], X[:-1]))
ym1 = np.concatenate((y[:-1], y[:-1], y[:-1]))

In [27]:
model.fit(Xm1, ym1, batch_size=32, epochs=25)

Epoch 1/25
330/330 [==============================] - 3s 8ms/step - loss: 57.7723 - acc: 0.0273
Epoch 2/25
330/330 [==============================] - 2s 6ms/step - loss: 52.4770 - acc: 0.1091
Epoch 3/25
330/330 [==============================] - 2s 6ms/step - loss: 48.9575 - acc: 0.2061
Epoch 4/25
330/330 [==============================] - 2s 6ms/step - loss: 45.8179 - acc: 0.2333
Epoch 5/25
330/330 [==============================] - 2s 6ms/step - loss: 42.9783 - acc: 0.1636
Epoch 6/25
330/330 [==============================] - 2s 6ms/step - loss: 40.5606 - acc: 0.1364
Epoch 7/25
330/330 [==============================] - 2s 6ms/step - loss: 38.4506 - acc: 0.0909
Epoch 8/25
330/330 [==============================] - 2s 6ms/step - loss: 36.5652 - acc: 0.0909
Epoch 9/25
330/330 [==============================] - 2s 7ms/step - loss: 34.9303 - acc: 0.1242
Epoch 10/25
330/330 [==============================] - 2s 6ms/step - loss: 33.4938 - acc: 0.1273
Epoch 11/25
330/330 [==================

In [28]:
evaluate = np.vectorize(lambda x, t: 1 if x > t else 0)

In [29]:
for t in np.linspace(0.01, 0.04, 31):
    x = model.predict(X[-1].reshape(1, X.shape[1]))
    y_pred = evaluate(x[0], t)
    print('thresh=%.3f:' % t, np.sum(y_pred))

thresh=0.010: 27
thresh=0.011: 22
thresh=0.012: 21
thresh=0.013: 19
thresh=0.014: 16
thresh=0.015: 15
thresh=0.016: 14
thresh=0.017: 14
thresh=0.018: 13
thresh=0.019: 12
thresh=0.020: 10
thresh=0.021: 10
thresh=0.022: 10
thresh=0.023: 9
thresh=0.024: 9
thresh=0.025: 9
thresh=0.026: 9
thresh=0.027: 9
thresh=0.028: 8
thresh=0.029: 8
thresh=0.030: 8
thresh=0.031: 8
thresh=0.032: 8
thresh=0.033: 7
thresh=0.034: 7
thresh=0.035: 7
thresh=0.036: 7
thresh=0.037: 6
thresh=0.038: 6
thresh=0.039: 6
thresh=0.040: 6


In [36]:
x = model.predict(X[-1].reshape(1, X.shape[1]))
y_pred = evaluate(x[0], 0.019)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [33]:
pred = pd.DataFrame(data=y_pred.reshape(1, y.shape[1]), columns=tune_frame.columns)

labels = tune_frame.columns.tolist()
labels = [l.split('_') for l in labels]
labels = [(int(l[0][1:]), int(l[1][1:])) for l in labels]
time = max([l[0] for l in labels])
notes = max([l[1] for l in labels])
midi = pd.DataFrame(index=range(time + 1), columns=range(notes + 1))

column_lookup = dict(zip(tune_frame.columns.tolist(), labels))

for k, v in column_lookup.items():
    val = pred.loc[:,k].values[0]
    midi.loc[v[0], v[1]] = val
    
midi.rename(columns=dict(zip(midi.columns.tolist(), notes_df.note.tolist())), inplace=True)
midi.to_csv('out.csv')

In [34]:
midi

,Bb3,B3,Cb4,C4,C#4,Db4,D4,D#4,Eb4,E4,...,E#5,Fb5,F5,F#5,Gb5,G5,G#5,Ab5,A5,A#5
0,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,1,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
